# ----------------------------------------------------------------------------------------------------

# VALIDATIONS (validations.py)

In [1]:
# LIBRARIES
import numpy as np
import pandas as pd
import yfinance as yf
import matplotlib.pyplot as plt
from functions import QAA
from datetime import datetime
from scipy.optimize import minimize
from scipy.spatial.distance import squareform
from scipy.cluster.hierarchy import linkage, dendrogram, leaves_list

In [2]:
tickers = ["ABBV", "MET", "OXY", "PERI"]
start_date = "2020-01-02"
end_date = "2024-01-23"

validation_data = yf.download(tickers, start=start_date, end=end_date)['Adj Close']
validation_returns = validation_data.pct_change().dropna()

c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
[                       0%%                      ]c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta 

# ----------------------------------------------------------------------------------------------------

## MIN VARIANCE

 ### functions.py

In [3]:
optimization_models = ["SLSQP", "MONTECARLO", "COBYLA"]#, "PSO"]

for optimization_model in optimization_models:
    qaa_instance = QAA(
        tickers=["ABBV", "MET", "OXY", "PERI"],
        benchmark="SPY",
        rf=0.02,
        lower_bound=0.10,
        higher_bound=0.90,
        start_date="2020-01-02",
        end_date="2024-01-23",
        expected_returns=np.array([.15, .1, .1, .1]),
        opinions=np.array([[1, 0, 0, 0], [0, 1, -3, 0], [0, 0, 1, -1], [0, 0, 0, 0]]),
        MAR=0.2,
        optimization_model=optimization_model,
        QAA_strategy="MIN VARIANCE"
    )
    
    try:
        data, returns, std, var, cov, corr = qaa_instance.assets_metrics()

        optimal_weights = qaa_instance.QAA_strategy_selection(returns=returns)
        
    except ValueError as ve:
        print(f"Error: {str(ve)}")

c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_erro


Optimal Portfolio Weights for MIN VARIANCE QAA using SLSQP optimization:
ABBV    0.25
MET     0.25
OXY     0.25
PERI    0.25
Name: Optimal Weights, dtype: float64


c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_erro


Optimal Portfolio Weights for MIN VARIANCE QAA using MONTECARLO optimization:
ABBV    0.656560
MET     0.168150
OXY     0.078238
PERI    0.097052
Name: Optimal Weights, dtype: float64

Optimal Portfolio Weights for MIN VARIANCE QAA using COBYLA optimization:
ABBV    0.7
MET     0.1
OXY     0.1
PERI    0.1
Name: Optimal Weights, dtype: float64


##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 ### validations.py

In [4]:
from pypfopt import EfficientFrontier
from pypfopt import expected_returns
from pypfopt import risk_models

mu = expected_returns.mean_historical_return(validation_data)
Sigma = risk_models.sample_cov(validation_data)

ef = EfficientFrontier(mu, Sigma)

ef.add_constraint(lambda w: w >= 0.10)
ef.add_constraint(lambda w: w <= 0.9)

ef.min_volatility()

OrderedDict([('ABBV', 0.7), ('MET', 0.1), ('OXY', 0.1), ('PERI', 0.1)])

# ----------------------------------------------------------------------------------------------------

## MAX SHARPE RATIO

 ### functions.py

In [5]:
optimization_models = ["SLSQP", "MONTECARLO", "COBYLA"]#, "PSO"]

for optimization_model in optimization_models:
    qaa_instance = QAA(
        tickers=["ABBV", "MET", "OXY", "PERI"],
        benchmark="SPY",
        rf=0.02,
        lower_bound=0.10,
        higher_bound=0.90,
        start_date="2020-01-02",
        end_date="2024-01-23",
        expected_returns=np.array([.15, .1, .1, .1]),
        opinions=np.array([[1, 0, 0, 0], [0, 1, -3, 0], [0, 0, 1, -1], [0, 0, 0, 0]]),
        MAR=0.2,
        optimization_model=optimization_model,
        QAA_strategy="MAX SHARPE RATIO"
    )
    
    try:
        data, returns, std, var, cov, corr = qaa_instance.assets_metrics()

        optimal_weights = qaa_instance.QAA_strategy_selection(returns=returns)
        
    except ValueError as ve:
        print(f"Error: {str(ve)}")

c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_erro


Optimal Portfolio Weights for MAX SHARPE RATIO QAA using SLSQP optimization:
ABBV    0.1
MET     0.1
OXY     0.7
PERI    0.1
Name: Optimal Weights, dtype: float64


c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_error_hours, 'h')
c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\yfinance\utils.py:775: FutureWarning: The 'unit' keyword in TimedeltaIndex construction is deprecated and will be removed in a future version. Use pd.to_timedelta instead.
  df.index += _pd.TimedeltaIndex(dst_erro


Optimal Portfolio Weights for MAX SHARPE RATIO QAA using MONTECARLO optimization:
ABBV    0.084963
MET     0.136021
OXY     0.686987
PERI    0.092029
Name: Optimal Weights, dtype: float64

Optimal Portfolio Weights for MAX SHARPE RATIO QAA using COBYLA optimization:
ABBV    0.9
MET     0.9
OXY     0.9
PERI    0.9
Name: Optimal Weights, dtype: float64


##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 ### validations.py

In [6]:
from pypfopt import EfficientFrontier
from pypfopt import expected_returns
from pypfopt import risk_models

mu = expected_returns.mean_historical_return(validation_data)
Sigma = risk_models.sample_cov(validation_data)

ef = EfficientFrontier(mu, Sigma)

ef.add_constraint(lambda w: w >= 0.10)
ef.add_constraint(lambda w: w <= 0.9)

ef.max_sharpe()

OrderedDict([('ABBV', 0.5701564412846583),
             ('MET', 0.1),
             ('OXY', 0.1),
             ('PERI', 0.2298435587153416)])

# ----------------------------------------------------------------------------------------------------

## OMEGA

 ### functions.py

##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 ### validations.py

# ----------------------------------------------------------------------------------------------------

## SEMIVARIANCE

 ### functions.py

##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 ### validations.py

# ----------------------------------------------------------------------------------------------------

## SORTINO RATIO

 ### functions.py

##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 ### validations.py

# ----------------------------------------------------------------------------------------------------

## BLACK LITTERMAN

 ### functions.py

##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 ### validations.py

# ----------------------------------------------------------------------------------------------------

## ROY SAFETY RATIO

 ### functions.py

##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 ### validations.py

# ----------------------------------------------------------------------------------------------------

## MARTINGALE

 ### functions.py

##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 ### validations.py

# ----------------------------------------------------------------------------------------------------

## HRP

 ### functions.py

##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 ### validations.py

In [7]:
from pypfopt.hierarchical_portfolio import HRPOpt

HRPOpt(validation_returns).optimize()

c:\Users\diego\OneDrive - ITESO\ITESO - TITA\Semestres - TITA\10mo Sem -TITA\PAP\PAP\venvPAP\Lib\site-packages\pypfopt\hierarchical_portfolio.py:138: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value '[0.7259991 0.7259991]' has dtype incompatible with int64, please explicitly cast to a compatible dtype first.
  w[first_cluster] *= alpha  # weight 1


OrderedDict([('ABBV', 0.6312962175894649),
             ('MET', 0.2110907364293606),
             ('OXY', 0.06291016430236329),
             ('PERI', 0.09470288167881123)])

# ----------------------------------------------------------------------------------------------------

## ten

 ### functions.py

##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 ### validations.py

# ----------------------------------------------------------------------------------------------------

## eleven

 ### functions.py

##### ~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~~

 ### validations.py